In [ ]:
import torch                       # pytorch
import torch.nn as nn              # pytorch for neural network
import numpy as np                 # for algebric functions
import matplotlib.pyplot as plt    # to plot graph
import torchvision
import torch.optim as optim
import copy
import random
import time
import gc
import pickle
from torch.utils.data.sampler import SubsetRandomSampler
%matplotlib inline

torch.set_default_tensor_type('torch.cuda.FloatTensor')
dtype = dtype = torch.cuda.FloatTensor
device = torch.device("cuda:0")
print("Device :", device)

Device : cuda:0


In [ ]:
trainset = torchvision.datasets.MNIST(train=True,download=True,root= "./mnist",
                                     transform= torchvision.transforms.ToTensor())
# trainset, valset = torch.utils.data.random_split(dataset=dataset, lengths=[45000, 5000], generator=torch.cuda.manual_seed(123))
testset = torchvision.datasets.MNIST(train=False,download=True,root= "./mnist",
                                     transform= torchvision.transforms.ToTensor())
print(len(trainset))
# print(len(valset))
print(len(testset))

60000
10000


In [ ]:
#Hyperparams
num_clients = 100
label_to_clients = 20
batch_size = 10
num_epochs = 5
lr = 0.5

# Skewing the data


In [ ]:
targs = np.array(trainset.targets)
idx = {}
idx_val = {}
for i in range(10):
    idx[i] = np.where(targs == i)[0]
    print(len(idx[i]))

5923
6742
5958
6131
5842
5421
5918
6265
5851
5949


# Let each class be present in $label_to_clients clients, clients chosen randomly

In [ ]:
cmap = {}
data = {}
vald = {}
for i in range(num_clients):
    data[i] = []
    vald[i] = []

#Data from the skew notebook

#highly non-iid data

client_num = 0
i = 0
while i < 10:
    cmap[i] = []
    cmap[i+1] = []
    for j in range((2 * num_clients)//10):
        cmap[i].append(client_num)
        if i < 9 :
            cmap[i + 1].append(client_num)
        client_num += 1
    i += 2

#iid data
# for i in range(10):
#     cmap[i] = []
#     for j in range(num_clients):
#         cmap[i].append(j)

for i in range(10):
    print (cmap[i])

for i in range(10):
    split = int(np.floor(1/label_to_clients * len(idx[i])))
    for j in range(label_to_clients):
        data[cmap[i][j]].extend(idx[i][j * split : (j + 1) * split])
for i in range(num_clients):
    np.random.shuffle(data[i])
    data[i], vald[i] = data[i][ : (int)(0.9 *len(data[i]))], data[i][(int)(0.9 * len(data[i])) : ]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
[40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
[40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
[60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79]
[60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79]
[80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
[80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [ ]:
freq = []

for i in range(num_clients):
    # print(len(data[i]))
    # print(labels[data[i]])
    # data[i] = data[i][1:300] + data[i][5000:5300]
    unique, counts = np.unique(np.array(targs[data[i]]), return_counts= True)
    print("Client ", i, " : ")
    print(np.asarray((unique, counts)).T)
    freq.append(np.sum(counts)/len(trainset))
    unique, counts = np.unique(np.array(targs[vald[i]]), return_counts= True)
    print("Val Client ", i, " : ")
    print(np.asarray((unique, counts)).T)
print("Percentage of points in the clients : ", freq)

Client  0  : 
[[  0 265]
 [  1 304]]
Val Client  0  : 
[[ 0 31]
 [ 1 33]]
Client  1  : 
[[  0 266]
 [  1 303]]
Val Client  1  : 
[[ 0 30]
 [ 1 34]]
Client  2  : 
[[  0 266]
 [  1 303]]
Val Client  2  : 
[[ 0 30]
 [ 1 34]]
Client  3  : 
[[  0 265]
 [  1 304]]
Val Client  3  : 
[[ 0 31]
 [ 1 33]]
Client  4  : 
[[  0 262]
 [  1 307]]
Val Client  4  : 
[[ 0 34]
 [ 1 30]]
Client  5  : 
[[  0 265]
 [  1 304]]
Val Client  5  : 
[[ 0 31]
 [ 1 33]]
Client  6  : 
[[  0 267]
 [  1 302]]
Val Client  6  : 
[[ 0 29]
 [ 1 35]]
Client  7  : 
[[  0 268]
 [  1 301]]
Val Client  7  : 
[[ 0 28]
 [ 1 36]]
Client  8  : 
[[  0 262]
 [  1 307]]
Val Client  8  : 
[[ 0 34]
 [ 1 30]]
Client  9  : 
[[  0 265]
 [  1 304]]
Val Client  9  : 
[[ 0 31]
 [ 1 33]]
Client  10  : 
[[  0 265]
 [  1 304]]
Val Client  10  : 
[[ 0 31]
 [ 1 33]]
Client  11  : 
[[  0 268]
 [  1 301]]
Val Client  11  : 
[[ 0 28]
 [ 1 36]]
Client  12  : 
[[  0 269]
 [  1 300]]
Val Client  12  : 
[[ 0 27]
 [ 1 37]]
Client  13  : 
[[  0 266]
 [  1 

# Split data equally

In [ ]:
# num_train = len(trainset)
# indices = list(range(num_train))
# np.random.shuffle(indices)
# split = int(np.floor(1/num_clients * num_train))
# data = []
# for i in range(num_clients):
#     data.append(indices[i * split : (i + 1) * split])
#     print(len(data[i]))

In [ ]:
# cfg = {
#     'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
#     'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
#     'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
#     'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
# }


# class VGG(nn.Module):
#     def __init__(self, vgg_name):
#         super(VGG, self).__init__()
#         self.features = self._make_layers(cfg[vgg_name])
#         self.classifier = nn.Linear(512, 10)

#     def forward(self, x):
#         out = self.features(x)
#         out = out.view(out.size(0), -1)
#         out = self.classifier(out)
#         return out

#     def _make_layers(self, cfg):
#         layers = []
#         in_channels = 3
#         for x in cfg:
#             if x == 'M':
#                 layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
#             else:
#                 layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
#                            nn.ReLU(inplace=True),
#                            nn.BatchNorm2d(x)]
#                 in_channels = x
#         layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
#         return nn.Sequential(*layers)


In [ ]:
avg_model = nn.Sequential(
        nn.Conv2d(kernel_size=5, out_channels=32, in_channels=1),
        nn.MaxPool2d(kernel_size=2),
        nn.Conv2d(kernel_size=5, out_channels=64, in_channels=32),
        nn.MaxPool2d(kernel_size=2),
        nn.Flatten(),
        nn.Linear(in_features=1024, out_features=512),
        nn.ReLU(),
        nn.Linear(in_features=512, out_features=10),
        nn.Softmax(dim=1)
).to(device)

from torchsummary import summary
summary(avg_model, (1, 28, 28))

In [ ]:
criterion = nn.CrossEntropyLoss()
server_updates = 1
train_accuracy = 0
test_accuracy = 0
# fdata = torch.utils.data.DataLoader(dataset=trainset, batch_size=64)
testdata = torch.utils.data.DataLoader(dataset=testset, batch_size=64)
# for iter, batch in enumerate(fdata):
#         inputs, labels = batch
#         inputs, labels = inputs.to(device), labels.to(device)
#         outputs = avg_model(inputs)
#         _, predicted = torch.max(outputs.data, 1)
#         correct_pred = predicted.eq(labels.data).sum().item()
#         train_accuracy = train_accuracy + (correct_pred/64) * 100
#         del inputs, labels, outputs, correct_pred, predicted
# init_serv_acc = train_accuracy * 64/len(trainset)
for iter, batch in enumerate(testdata):
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = avg_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        correct_pred = predicted.eq(labels.data).sum().item()
        test_accuracy = test_accuracy + (correct_pred/64) * 100
        del inputs, labels, outputs, correct_pred, predicted
init_serv_test_acc = test_accuracy * 64/len(testset)
# print("Initial server train accuracy ", init_serv_acc)
print("Initial server test accuracy ", init_serv_test_acc)


In [ ]:
train_acc_list = []
test_acc_list = []
save_model = "./results/synch_update_" + str(num_clients) + "," + str(num_epochs) + "," + str(batch_size) + ".sav"

while True:
    train_accuracy = 0
    test_accuracy = 0
    random_clients = []
    client_models = []
    for i in range(100):
        turn = np.random.randint(0,num_clients)
        while turn in random_clients:
            turn = np.random.randint(0,num_clients)
        random_clients.append(turn)
    print("Training ", random_clients, " clients")
    for turn in random_clients:
    #client train
        print("Client ", turn, " has started training with batch size ", batch_size, " for ", num_epochs, " epochs")
        local_net = copy.deepcopy(avg_model)
        local_net.to(device)
        # optimizer = optim.Adam(local_net.parameters())
        optimizer = optim.SGD(local_net.parameters(), lr = lr)
        val_data = torch.utils.data.DataLoader(dataset=trainset, batch_size=min(64, len(vald[turn])), sampler = SubsetRandomSampler(vald[turn]))
        for epoch in range(num_epochs):
            # !nvidia-smi
            train_data = torch.utils.data.DataLoader(trainset, batch_size, sampler = SubsetRandomSampler(data[turn]))
            running_loss = 0
            train_accuracy = 0
            val_accuracy = 0
            for iter, batch in enumerate(train_data):
                inputs, labels = batch
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = local_net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                correct_pred = predicted.eq(labels.data).sum().item()
                train_accuracy += (correct_pred/batch_size) * 100
                del outputs, loss
            # if epoch % 5 == 0:
            print("Client ", turn, " epoch : ", epoch, " loss : ", running_loss/(len(data[turn])/batch_size))
            print("Train Accuracy : ", train_accuracy/(len(data[turn])/batch_size))
            for iter2, batch2 in enumerate(val_data):
                inputs, labels = batch2
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = local_net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                correct_pred = predicted.eq(labels.data).sum().item()
                val_accuracy += (correct_pred/min(64, len(vald[turn]))) * 100
                del outputs
            print("Validation accuracy : ", val_accuracy * min(64, len(vald[turn]))/len(vald[turn]))
        
        client_models.append(copy.deepcopy(local_net))
        del local_net
   
    
    #Update server
    for name1, param1 in avg_model.named_parameters():
        param1.data.copy_(torch.zeros(param1.shape))

    print("#models --> ", len(client_models))
    for c in client_models:
        for name1, param1 in c.named_parameters():
            for name2, param2 in avg_model.named_parameters():
                if name1 == name2:
                    param2.data.copy_(param2.data + param1.data/100)
    
    # for iter, batch in enumerate(fdata):
    #     inputs, labels = batch
    #     inputs, labels = inputs.to(device), labels.to(device)
    #     outputs = avg_model(inputs)
    #     _, predicted = torch.max(outputs.data, 1)
    #     correct_pred = predicted.eq(labels.data).sum().item()
    #     train_accuracy += (correct_pred/64) * 100
    #     del outputs
    # print("Current server train accuracy : ", train_accuracy * 64/len(trainset))
    # print("Increase in server train acc : ", train_accuracy * 64/len(trainset) - init_serv_acc, " after ", server_updates, " server updates")
    # train_acc_list.append(train_accuracy * 64/len(trainset))
    for iter, batch in enumerate(testdata):
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = avg_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        correct_pred = predicted.eq(labels.data).sum().item()
        test_accuracy += (correct_pred/64) * 100
        del outputs
    print("Current server test accuracy : ", test_accuracy * 64/len(testset))
    print("Increase in server test acc : ", test_accuracy * 64/len(testset) - init_serv_test_acc, " after ", server_updates, " server updates")
    test_acc_list.append(test_accuracy * 64/len(testset))
    fig, ax = plt.subplots()
    ax.set_xlabel('# Communication rounds')
    ax.set_ylabel('# Accuracy')
    # ax.plot(list(range(server_updates)), train_acc_list)
    ax.plot(list(range(server_updates)), test_acc_list, label = 'Test Accuracy')
    save_dir = "./results/synch_update_" + str(num_clients) + "," + str(num_epochs) + "," + str(batch_size) + ".png"
    plt.savefig(save_dir)
    plt.show()
    server_updates += 1
    pickle.dump(avg_model, open(save_model, 'wb'))
    lr *= 0.92
